In [5]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain
import os
os.environ["OPENAI_API_KEY"] = '<open-ai-key>'

## Simple Chain

In [6]:
llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["category"],
    template="What is the best online platform for {category}?",
)
chain = LLMChain(llm=llm, prompt=prompt, output_key="platforms")
print(chain.run("tech professionals"))



The best online platform for tech professionals would depend on their specific needs and interests. Popular online platforms for tech professionals include Stack Overflow, GitHub, Reddit, Slack, Quora, and LinkedIn.


### Simple Sequential Chain

In [7]:
feature_template = """Tell me the feature of the platforms mentioned here in {platforms}"""
feature_prompt_template = PromptTemplate(input_variables=["platforms"], template=feature_template)
feature_chain = LLMChain(llm=llm, prompt=feature_prompt_template)
overall_chain = SimpleSequentialChain(chains=[chain, feature_chain], verbose=True)
print(overall_chain.run("tech professionals"))



> Entering new SimpleSequentialChain chain...


The best online platform for tech professionals is LinkedIn. It is designed to help professionals network and find jobs, and it offers a platform for users to showcase their technical skills and experience. Additionally, LinkedIn has numerous groups and forums dedicated to tech professionals, providing an excellent resource to share knowledge and collaborate.
 

Other features of LinkedIn include:

• Profile Building - Create a comprehensive profile that highlights your professional experience, certifications, and education.

• Job Board - Search for job openings in the tech industry.

• Groups and Forums - Join groups and forums related to your field of expertise and network with other professionals.

• Events - See what events are occurring in the tech world.

• Company Directory - Search for tech companies and find more information about their products and services.

• InMail - Reach out to contacts directly without knowing their ema

### Sequential Chain

In [8]:
feature_template = """Tell me the feature of the platforms mentioned here in {platforms}"""
feature_prompt_template = PromptTemplate(input_variables=["platforms"], template=feature_template)
feature_chain = LLMChain(llm=llm, prompt=feature_prompt_template, output_key="features")
overall_chain = SequentialChain(
    chains=[chain, feature_chain],
    input_variables=["category"],
    output_variables=["platforms", "features"],
    verbose=True)
print(overall_chain("tech professionals"))



> Entering new SequentialChain chain...

> Finished chain.
{'category': 'tech professionals', 'platforms': '\n\nThe best online platform for tech professionals is probably Stack Overflow. It is a Q&A platform that allows developers to ask and answer technical questions, share code snippets, and collaborate with other professionals in the industry. It also has an extensive knowledge base and tutorials to help professionals stay up-to-date on the latest technologies.', 'features': '\n\nSome of the features of Stack Overflow include:\n\n• A comprehensive Q&A platform with a variety of topics\n• Ability to ask, answer and collaborate with other tech professionals\n• Ability to share code snippets with others in the community\n• Extensive knowledge base and tutorials to help stay current\n• Ability to create a profile and earn reputation points for doing well\n• Ability to follow and search for topics \n• Ability to filter content by tags\n• Ability to flag and report spam, fraud, or offe

### OpenAI Function Chain

In [10]:
from typing import Optional

from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [11]:
from langchain.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Identifying information about a person."""

    name: str = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")
    fav_food: Optional[str] = Field(None, description="The person's favorite food")

In [12]:
# If we pass in a model explicitly, we need to make sure it supports the OpenAI function-calling API.
llm = ChatOpenAI(model="gpt-4", temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a world class algorithm for extracting information in structured formats."),
        ("human", "Use the given format to extract information from the following input: {input}"),
        ("human", "Tip: Make sure to answer in the correct format"),
    ]
)

chain = create_structured_output_chain(Person, llm, prompt, verbose=True)
chain.run("Adam is 30 and fond of pastries")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a world class algorithm for extracting information in structured formats.
Human: Use the given format to extract information from the following input: Adam is 30 and fond of pastries
Human: Tip: Make sure to answer in the correct format

> Finished chain.


Person(name='Adam', age=30, fav_food='pastries')